# 第6章  数据载入、存储及文件格式

In [1]:
import pandas as pd
import numpy as np
import os
import sys

## 6.1 文本格式数据的读写

### <font color="#4876FF">exsample one</font>

In [2]:
# ex1.csv原始状态
!cat pydata-book-2nd-edition/examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
# 使用read_csv读取csv文件
df = pd.read_csv('pydata-book-2nd-edition/examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# 使用read_table读取csv文件
pd.read_table('pydata-book-2nd-edition/examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### <font color="#4876FF">exsample two</font>

In [5]:
# ex2.csv原始状态
!cat pydata-book-2nd-edition/examples/ex2.csv
# ex2.csv没有表头，要读取该文件,你需要选择一些选项。你可以允许pandas自动分配默认列名,也可以自己指定列名。

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
# 使用read_csv读取没有表头的csv文件，这样pandas会自动分配默认列名
pd.read_csv('pydata-book-2nd-edition/examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# 使用read_csv读取没有表头的csv文件，手动添加表头
label = pd.Index(list('abcd') + ['message'])
pd.read_csv('pydata-book-2nd-edition/examples/ex2.csv', names=label)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# 使用read_csv读取没有表头的csv文件，手动添加表头
# 假设你想要message列成为返回DataFrame的索引,你可以指定位置4的列为索引,或将'message'传给参数index_col:
label = pd.Index(list('abcd') + ['message'])
pd.read_csv('pydata-book-2nd-edition/examples/ex2.csv', names=label, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


### <font color="#4876FF">exsample three</font>

In [9]:
# csv_mindex.csv原始状态
!cat pydata-book-2nd-edition/examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
# 当你想要从多个列中形成一个分层索引,需要传入一个包含列序号或列名的列表
parsed = pd.read_csv('pydata-book-2nd-edition/examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

### <font color="#4876FF">exsample four</font>

In [11]:
# ex3.txt原始状态
with open('pydata-book-2nd-edition/examples/ex3.txt', 'r') as f:
    text = list(f)
text

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [12]:
# 当字段是以多种不同数量的空格分开时,尽管你可以手工处理,
# 但在这些情况下也可以向read_table传入一个正则表达式作为分隔符。
# 在本例中,正则表达式为\s+,因此我们可以得到:
result = pd.read_table('pydata-book-2nd-edition/examples/ex3.txt', sep=r'\s+')
print('本例中,由于列名的数量比数据的列数少一个,因此read_table推断第一列应当作为DataFrame的索引。')
result

本例中,由于列名的数量比数据的列数少一个,因此read_table推断第一列应当作为DataFrame的索引。


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


### <font color="#4876FF">exsample five</font>

In [13]:
# ex4.csv原始状态
!cat pydata-book-2nd-edition/examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [14]:
# 解析函数有很多附加参数帮助你处理各种发生异常的文件格式(表6-2列举了一部分)。
# 例如,你可以使用skiprows来跳过第一行、第三行和第四行:
pd.read_csv('pydata-book-2nd-edition/examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### <font color="#4876FF">exsample six</font>

In [15]:
# ex5.csv原始状态
!cat pydata-book-2nd-edition/examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [16]:
# 缺失值处理是文件解析过程中一个重要且常常微妙的部分。
# 通常情况下,缺失值要么不显示(空字符串),要么用一些标识值。
# 默认情况下,pandas使用一些常见的标识,例如NA和NULL:
result = pd.read_csv('pydata-book-2nd-edition/examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [18]:
# na_values选项可以传入一个列表或一组字符串来处理缺失值:
result = pd.read_csv('pydata-book-2nd-edition/examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
# 在字典中,每列可以指定不同的缺失值标识:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('pydata-book-2nd-edition/examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 6.1.1 分块读入文本文件

### <font color="#4876FF">exsample one</font>

In [20]:
result = pd.read_csv('pydata-book-2nd-edition/examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [21]:
# 如果你只想读取一小部分行(避免读取整个文件),可以指明nrows:
result = pd.read_csv('pydata-book-2nd-edition/examples/ex6.csv', nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [22]:
# 为了分块读入文件,可以指定chunksize作为每一块的行数:
chunker = pd.read_csv('pydata-book-2nd-edition/examples/ex6.csv', chunksize=1000)
chunker

In [23]:
# read_csv返回的TextParser对象允许你根据chunksize遍历文件。
# 例如,我们可以遍历ex6.csv,并对'key'列聚合获得计数值:
chunker = pd.read_csv("pydata-book-2nd-edition/examples/ex6.csv", chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot.iloc[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 6.1.2 将数据写入文本格式

### <font color="#4876FF">exsample one</font>

In [24]:
# 数据可以导出为分隔的形式。让我们看下之前读取的CSV文件:
data = pd.read_csv('pydata-book-2nd-edition/examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [25]:
# 使用DataFrame的to_csv方法,我们可以将数据导出为逗号分隔的文件:
data.to_csv("examples/out.csv")
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [26]:
# 当然,其他的分隔符也是可以的(写入到sys.stdout时,控制台中打印的文本结果):
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [27]:
# 缺失值在输出时以空字符串出现。你也许想要用其他标识值对缺失值进行标注:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [28]:
# 如果没有其他选项被指定的话,行和列的标签都会被写入。不过二者也都可以禁止写入:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [29]:
# 你也可以仅写入列的子集,并且按照你选择的顺序写入:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### <font color="#4876FF">exsample two</font>

In [30]:
# Series也有to_csv方法:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## 6.1.3 使用分隔格式

### <font color="#4876FF">exsample one</font>

In [31]:
# ex7.csv原始状态
!cat pydata-book-2nd-edition/examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [32]:
# 对于任何带有单字符分隔符的文件,你可以使用Python的内建csv模块。
# 要使用它,需要将任一打开的文件或文件型对象传给csv.reader:
import csv

ex7_path = 'pydata-book-2nd-edition/examples/ex7.csv'

with open(ex7_path, 'r') as f:
    reader = csv.reader(f)
    # 像遍历文件那样遍历reader会产生元组,元组的值为删除了引号的字符:
    for line in reader:
        print(line)
        
# 之后,你就可以自行做一些必要处理,以将数据整理为你需要的形式。
# 让我们按部就班,首先将文件读取为行的列表:
with open(ex7_path, 'r') as f:
    lines = list(csv.reader(f))
header, values = lines[0], lines[1:]
print('Header is :', header)
print('Values is :', values)

# 再然后,我们使用字典推导式和表达式zip(*values)生成一个包
# 含数据列的字典,字典中行转置成列:
data_dict = {h: v for h, v in zip(header, zip(*values))}
print('data_dict is:', data_dict)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']
Header is : ['a', 'b', 'c']
Values is : [['1', '2', '3'], ['1', '2', '3']]
data_dict is: {'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}


### <font color="#4876FF">exsample two</font>

CSV文件有多种不同风格。  
如需根据不同的分隔符、字符串引用约定或行终止符定义一种新的格式时,我们可以使用csv.Dialect定义一个简单的子类:
```python
class my_dialect(csv.Dialect):
    lineterminator = '\n' # 行终止符号
    delimiter = ';'       # 分隔符
    quotechar = '"'       # 引用符
    quoting = csv.QUOTE_MINIMAL
    
reader = csv.reader(f, dialect=my_dialect)
```

我们也可以不必定义子类,直接将CSV方言参数传入csv.reader的关键字参数:
```python
reader = csv.reader(f, delimiter='|')
```

需要手动写入被分隔的文件时,你可以使用csv.writer。  
这个函数接收一个已经打开的可写入文件对象以及和csv.reader相同的CSV方言、格式选项:
```python
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))
```

## 6.1.4 JSON数据

### <font color="#4876FF">exsample one</font>

In [35]:
obj = """
    {"name": "Wes",
    "places_lived": ["United States", "Spain", "Germany"],
    "pet": null,
    "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
    {"name": "Katie", "age": 38,
    "pets": ["Sixes", "Stache", "Cisco"]}]
    }
    """

import json

# 将JSON字符串转换为Python形式时,使用json.loads方法:
result = json.loads(obj)
print('result is:', result)

# json.dumps可以将Python对象转换回JSON:
asjson = json.dumps(result)

# 你将自行决定如何将JSON对象或对象列表转换为DataFrame或其他数据结构。
# 比较方便的方式是将字典构成的列表(之前是JSON对象)传入DataFrame构造函数,并选出数据字段的子集:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

result is: {'name': 'Wes', 'places_lived': ['United States', 'Spain', 'Germany'], 'pet': None, 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']}, {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}


,name,age
0,Scott,30
1,Katie,38


In [43]:
# pandas.read_json可以自动将JSON数据集按照指定次序转换为Series或DataFrame。例如:
!cat pydata-book-2nd-edition/examples/example.json

# pandas.read_json的默认选项是假设JSON数组中的每个对象是表里的一行:
data = pd.read_json('pydata-book-2nd-edition/examples/example.json')
print('data:\n', data)

# 如果你需要从pandas中将数据导出为JSON,一种办法是对Series和DataFrame使用to_json方法:
print('data_to_json:', data.to_json())
print('data_to_json records module:', data.to_json(orient='records'))

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]
data:
    a  b  c
0  1  2  3
1  4  5  6
2  7  8  9
data_to_json: {"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
data_to_json records module: [{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## 6.1.5 XML和HTML:网络抓取

### <font color="#4876FF">exsample one</font>

In [48]:
tables = pd.read_html('pydata-book-2nd-edition/examples/fdic_failed_bank_list.html')
print('tables len:', len(tables))

# pandas.read_html函数有很多选项,但是默认情况下,
# 它会搜索并尝试解析所有包含在<table>标签中的表格型数据,返回的结果是DataFrame对象的列表:
failures = tables[0]
failures.head()

tables len: 1


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [51]:
# 因为failures有很多列,pandas在行内插入了换行符\。
# 你会在后续章节中学到,此处我们可以着手一些数据清理和分析工作,比如计算每年银行倒闭的数量:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64